# **Step Tracker Key Features**

## **1. Full-Stack Implementation**
![wtv](https://i.postimg.cc/sxcGHMhZ/image-2025-03-01-205721116.png)
- The Step Tracker feature integrates **frontend**, **backend**, and a **database** 
- Uses **Flask (Python)** for backend API handling.
- Uses **JavaScript** for frontend interaction.
- Stores data in a **relational database** using **SQLAlchemy**.

---

## **2. Backend API (Flask)**
- Provides a REST API using **Flask** and **Flask-RESTful**.
- Implements **JWT authentication** to secure API requests.
- Supports **CRUD operations** on step data:
  - **POST** `/api/steps` → Create a new step entry.
  - **GET** `/api/steps` → Retrieve user’s step data.
  - **PUT** `/api/steps` → Update an existing step entry.
  - **DELETE** `/api/steps` → Remove step entry.
```python
 @token_required()
        def post(self):
            """
            Add a new steps entry for the authenticated user.
            """
            current_user = g.current_user
            body = request.get_json()

            # Validate steps
            steps = body.get('steps')
            if steps is None or not isinstance(steps, int) or steps < 0:
                return {'message': 'Invalid steps provided'}, 400

            try:
                # Ensure no existing entry exists
                existing_steps = Steps.query.filter_by(user=current_user.uid).first()
                if existing_steps:
                    return {'message': 'Steps entry already exists'}, 400

                # Create a new steps entry
                new_steps = Steps(user=current_user.uid, steps=steps)
                new_steps.create()
                return jsonify({'message': 'Steps added successfully', 'steps': new_steps.read()})
            except Exception as e:
                return {'message': 'Failed to create steps', 'error': str(e)}, 500

        @token_required()
        def put(self):
            """
            Update an existing steps entry for the authenticated user.
            """
            current_user = g.current_user
            body = request.get_json()

            # Validate steps
            steps = body.get('steps')
            if steps is None or not isinstance(steps, int) or steps < 0:
                return {'message': 'Invalid steps provided'}, 400

            try:
                # Fetch and update the user's steps entry
                steps_entry = Steps.query.filter_by(user=current_user.uid).first()
                if not steps_entry:
                    return {'message': 'No steps entry found to update'}, 404

                steps_entry.steps = steps
                steps_entry.create()  # Save changes to the database
                return jsonify({'message': 'Steps updated successfully', 'steps': steps_entry.read()})
            except Exception as e:
                return {'message': 'Failed to update steps', 'error': str(e)}, 500

        @token_required()
        def get(self):
            """
            Get the current user's steps entry.
            """
            current_user = g.current_user

            try:
                steps_entry = Steps.query.filter_by(user=current_user.uid).first()
                if steps_entry:
                    return jsonify({'message': 'Steps retrieved successfully', 'steps': steps_entry.read()})
                else:
                    return {'message': 'No steps entry found for the user'}, 404
            except Exception as e:
                return {'message': 'Failed to retrieve steps', 'error': str(e)}, 500
        @token_required()
        def delete(self):
            """
            Delete an existing steps entry for the authenticated user.
            """
            current_user = g.current_user

            try:
                # Fetch and delete the user's steps entry
                steps_entry = Steps.query.filter_by(user=current_user.uid).first()
                if not steps_entry:
                    return {'message': 'No steps entry found to delete'}, 404

                steps_entry.delete()  # Delete the entry from the database
                return jsonify({'message': 'Steps entry deleted successfully'})
            except Exception as e:
                return {'message': 'Failed to delete steps entry', 'error': str(e)}, 500
    ```
---

## **3. Database Management (SQLAlchemy)**

- Stores step entries with the following attributes:
  - `id`: Unique identifier (Primary Key).
  - `user`: Username associated with steps.
  - `steps`: Number of recorded steps.

```python 
class Steps(db.Model):
   
    __tablename__ = 'steps'

    id = db.Column(db.Integer, primary_key=True)
    user = db.Column(db.String(255), nullable=False)
    steps = db.Column(db.Integer, nullable=False)

    def __init__(self, user, steps):
        self.user = user
        self.steps = steps
```
- Implements **database initialization, restore, and backup**:
  - `initSteps()`: Initializes the database with test data.
  ```python
  def initSteps():
    
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        tester_data = [
            Steps(user='user1', steps=5000),
            Steps(user='user2', steps=7500),
            Steps(user='user3', steps=12000)
        ]
        
        for data in tester_data:
            try:
                db.session.add(data)
                db.session.commit()
                print(f"Record created: {repr(data)}")
            except Exception as e:
                db.session.rollback()
    ```
  - `restore()`: Clears database and restores data from a backup.
  ```python
  @staticmethod
    def restore(data):
        with app.app_context():
            db.session.query(Steps).delete()
            db.session.commit()

            restored_steps = {}
            for steps_data in data:
                steps = Steps(
                    user=steps_data['user'],
                    steps=steps_data['steps']
                )
                steps.create()
                restored_steps[steps_data['id']] = steps

            return restored_steps
            ```

---

## **4. Frontend (JavaScript & HTML)**
- **User Interface (UI)**:
  - Provides an input field for entering step count.
  - Includes buttons for **Create, Update, Delete, and Fetching steps**.
- **Dynamic Updates using JavaScript**:
  - **Event Listeners** for button clicks.
  - **DOM Manipulation** to update displayed step data.
  - **Motivational Messages** based on step count.

---

## **5. Fetch API for Server Communication**
- Uses **JavaScript Fetch API** to interact with the backend.
- Ensures **secure requests** with **JWT authentication**.
- Fetching steps example:
  ```javascript
  async function fetchLastSteps() {
      const response = await fetch('/api/steps', {
          method: 'GET',
          credentials: 'include' // Sends authentication token
      });
      const data = await response.json();
      document.getElementById('steps-display').innerText = data.steps.steps;
  }
```